Prep

In [ ]:
import flywheel
import re
import sys
import os
import glob
import shutil
import subprocess
import docker
import pandas as pd
from tqdm import tqdm
from fw_heudiconv.cli import export

Initialise flywheel data:

In [ ]:
client = flywheel.Client()

RBC_proj = client.projects.find_first('label="ReproBrainChart"')

RBC_subjects = RBC_proj.subjects()

PNC_CS_proj = client.projects.find_first('label="PNC_CS_810336"')
PNC_LG_proj = client.projects.find_first('label="PNC_LG_810336"')

PNC_CS_subjects = PNC_CS_proj.subjects()
PNC_LG_subjects = PNC_LG_proj.subjects()

# get the penn_ids list
penn_ids = pd.read_csv('/storage/ttapera/RBC/data/bblid_renamed.csv', dtype = {'bblid': str, 'reid': str})

penn_ids['sub_id'] = 'sub-' + penn_ids['reid']

In [ ]:
# function to remove a subject from the RBC project
def remove_current_rbc_subject(row, RBC_subjects=RBC_subjects):

    target = [x for x in RBC_subjects if x.label == row['sub_id']]
    if len(target) == 1:
        print('found subject to remove: ', list(row), target[0].id)
        print('client.delete_subject(target[0].id)')
        return True
    else:
        return False


In [ ]:
penn_ids['process'] = penn_ids.apply(remove_current_rbc_subject, axis = 1)

In [ ]:
penn_ids.to_csv('/storage/ttapera/RBC/data/bblids_2.csv', index=False)

In [ ]:
sessions = None
#downloads = export.gather_bids(client, PNC_CS_proj.label, '80010', sessions)

In [ ]:
sessions = None

for index, row in tqdm(penn_ids.iterrows(), total=penn_ids.shape[0]):
    
    print("Processing...")
    should_replace = False
    # first remove the existing data on RBC
    should_replace = remove_current_rbc_subject(row, RBC_subjects)
    
    if should_replace:
        # download the data from PNC
        print("Gathering PNC CS BIDS...")
        pnc1_data = export.gather_bids(client, PNC_CS_proj.label, row['bblid'], sessions)        
        print("\nDownloading BIDS...\n")
        export.download_bids(
            client, pnc1_data, 
            '/storage/ttapera/RBC/data/', 
            folders_to_download = ['anat', 'func', 'fmap'],
            dry_run=False,
            name='PNC1')

        print("Gathering PNC LG BIDS...")
        try:
            pnc2_data = export.gather_bids(client, PNC_LG_proj.label, row['bblid'], sessions)

            export.download_bids(
                client, pnc2_data, 
                '/storage/ttapera/RBC/data/', 
                folders_to_download = ['anat', 'func', 'fmap'],
                dry_run=False,
                name='PNC2')
        except:
            print("no second session!")
        
        # merge any and all data
        print("n\Merging\n")
        os.system("cp -r -R /storage/ttapera/RBC/data/PNC*/* /storage/ttapera/RBC/data/upload/")
        
        # rename the files:
        print("\nRenaming\n")
        for root, subdirs, files in os.walk("/storage/ttapera/RBC/data/upload/"):

            for f in files:
                if row['bblid'] in f:
                    print("renaming file ", f)
                    new_name = f.replace(row['bblid'], row['reid'])
                    os.rename(root + "/" + f, root + "/" + new_name)

        # rename the subject-level directory
        old_name = "/storage/ttapera/RBC/data/upload/sub-" + row['bblid']
        new_name = old_name.replace(row['bblid'], row['reid'])
        shutil.move(old_name, new_name)        
        os.system("rm -rf /storage/ttapera/RBC/data/upload/sub*/sub*")
        
        # apply defacing here
        print("\nDefacing\n")
        for root, subdirs, files in os.walk("/storage/ttapera/RBC/data/upload/"):

            for f in files:
                if ".nii" in f:
                    print("defacing " + f)
                    cmd = ("docker run -ti --rm "
                           "-v {}:/data "
                           "pennbbl/mrideface "
                           "/data/{} "
                           "/opt/mrideface/talairach_mixed_with_skull.gca "
                           "/opt/mrideface/face.gca "
                           "/data/{} ".format(root, f, f))
                    
                    print(cmd)
                    output = subprocess.check_output(cmd)
        
        # upload here
        print("\nUploading\n")
        
        # remove from filesystem here
        print("\nClearing local data\n")
        os.system("rm -rf /storage/ttapera/RBC/data/upload/*")
        os.system("rm -rf /storage/ttapera/RBC/data/PNC*")
        os.system("rm -rf /storage/ttapera/RBC/data/sub*")